In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
XY_file_name = "200p_1000samples_XY.txt";
data_folder_path = "/ihome/hpark/zhf16/causalDeepVASE/data";
'''
Run MGM
Note: MGM was implemented in Java and the following Python APIs call the Java implementation.
Please restart the Python program after encountering a JVM problem.
The input data file should be ".txt" format and should also include the response variables.
Here is how the input data should look like:
X1 X2 ... Xp Y1 ... Yq
1  1  ... 1  1  ... 1
'''
from MGM.MGM import MGM
mgm = MGM();
mgm_output_file = mgm.runMGM(data_folder_path, XY_file_name,lambda_continuous_continuous = 0.3, lamda_continuous_discrete = 0.3, lamda_discrete_discrete = 0.3);
print("Please find MGM's output file as:");
mgm_output_file_path = data_folder_path+os.path.sep+mgm_output_file;
print(mgm_output_file_path);

/ihome/hpark/zhf16/causalDeepVASE/MGM/tetradLite.jar
Please find MGM's output file as:
/ihome/hpark/zhf16/causalDeepVASE/data/200p_1000samples_XY_MGM_associations.csv


In [4]:
X_file_name = "200p_1000samples_X.csv";
'''
#Generate knockoff data using one of three methods: ISEE Omega, DNN, and Cholesky_LU.
#Recommended: ISEE Omega or Cholesky_LU.
The code for generating ISEE Omega knockoff is implemented using R. Please make sure your computer has R installed.
'''
from DL.knockoff.KnockoffGenerator import KnockoffGenerator;
generator = KnockoffGenerator();

# knockoff_file_path = generator.Chol_Lu_knockoff(data_folder_path, X_file_name);

# #If want to generate ISEE Omega knockoff, please set the ISEE code path and R home environment.
# generator.set_ISEE_path("/ihome/hpark/zhf16/causalDeepVASE/");
# generator.set_R_home('/ihome/hpark/zhf16/.conda/envs/env36/lib/R');
# knockoff_file_path = generator.ISEE_knockoff(data_folder_path, X_file_name);

Y_file_name = '200p_1000samples_Y.csv';
knockoff_file_path = generator.DNN_knockoff(data_folder_path, X_file_name,Y_file_name);

print("The newly generated knockoff file is named as:")
print(knockoff_file_path);

Using TensorFlow backend.


__init__
(1000, 200)
(1000, 1)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               25728     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 36,097
Trainable params: 36,097
Non-trainable params: 0
_________________________________________________________________


/ihome/hpark/zhf16/causalDeepVASE/DL/knockoff/KnockoffGenerator.py:136: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, activation="relu", input_dim=200, kernel_initializer="glorot_normal", use_bias=True)`
  model.add(Dense(128, activation='relu', bias=True, init='glorot_normal', input_dim=n_features));
/ihome/hpark/zhf16/causalDeepVASE/DL/knockoff/KnockoffGenerator.py:137: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, activation="relu", kernel_initializer="glorot_normal", use_bias=True)`
  model.add(Dense(64, activation='relu', bias=True, init='glorot_normal'));
/ihome/hpark/zhf16/causalDeepVASE/DL/knockoff/KnockoffGenerator.py:138: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, activation="relu", kernel_initializer="glorot_normal", use_bias=True)`
  model.add(Dense(32, activation='relu', bias=True, init='glorot_normal'));
/ihome/hpark/zhf16/causalDeepVASE/DL/knockoff/KnockoffGenerator.py:139: UserWarning: Update your 

Epoch 1/50
1000/1000 [==============================] - 1s 555us/step - loss: 29795700.4880
Epoch 2/50
1000/1000 [==============================] - 0s 61us/step - loss: 29778715.5664
Epoch 3/50
1000/1000 [==============================] - 0s 58us/step - loss: 29730439.2740
Epoch 4/50
1000/1000 [==============================] - 0s 56us/step - loss: 29579986.6560
Epoch 5/50
1000/1000 [==============================] - 0s 54us/step - loss: 28926815.8520
Epoch 6/50
1000/1000 [==============================] - 0s 56us/step - loss: 27341376.4120
Epoch 7/50
1000/1000 [==============================] - 0s 53us/step - loss: 24028352.7240
Epoch 8/50
1000/1000 [==============================] - 0s 56us/step - loss: 20303100.6060
Epoch 9/50
1000/1000 [==============================] - 0s 55us/step - loss: 17187786.3780
Epoch 10/50
1000/1000 [==============================] - 0s 56us/step - loss: 14288449.8080
Epoch 11/50
1000/1000 [==============================] - 0s 55us/step - loss: 13651119.7

In [4]:
''''''
# After generating the knockoff data, run DNN
Y_file_name = '200p_1000samples_Y.csv';
X_knockoff_data = pd.read_csv(knockoff_file_path);
print(X_knockoff_data.shape)
# X_knockoff_data

#nutrient_data
original_data_Y = pd.read_csv(data_folder_path+os.path.sep+Y_file_name);
# original_data_Y

X_values = X_knockoff_data.values;
Y_values = original_data_Y.values;
    
p_num = int(X_values.shape[1] / 2);
n = X_values.shape[0];
print(X_values.shape);
print(Y_values.shape);
print(p_num);
    
X_origin = X_values[:, 0:p_num];
X_knockoff = X_values[:, p_num:];

x3D_train = np.zeros((n, p_num, 2));
x3D_train[:, :, 0] = X_origin;
x3D_train[:, :, 1] = X_knockoff;
label_train = Y_values;
    
coeff = 0.05 * np.sqrt(2.0 * np.log(p_num) / n);

n_outputs = original_data_Y.shape[1];

#Save the DNN output to the following directory.
result_dir = 'data/DNN_result/';
if not os.path.exists(result_dir):
    os.makedirs(result_dir);
    
from DL.DNN.DNN import DNN;
dnn = DNN();
model = dnn.build_DNN(p_num, n_outputs, coeff);
callback = DNN.Job_finish_Callback(result_dir,p_num);
dnn.train_DNN(model, x3D_train, label_train,callback);

(1000, 400)
(1000, 400)
(1000, 1)
200
__init__parameters
[layer]: Input	[shape]: [None, 200, 2] 

[layer]: LocallyConnected1D	[shape]: [None, 200, 1] 

[layer]: LocallyConnected1D	[shape]: [None, 200, 1] 

[layer]: Flatten	[shape]: [None, None] 

[layer]: Dense	[shape]: [None, 200] 

[layer]: Dense	[shape]: [None, 200] 

[layer]: Dense	[shape]: [None, 1] 

Epoch 1/20
1000/1000 [==============================] - 7s 7ms/step - loss: 29794096.5175
on_epoch_end
h_local1_weight = (200, 2, 1)
h_local2_weight = (200, 1, 1)
h0 = (200, 2)
h0_abs = (200, 2)
h1 = (200, 200)
h2 = (200, 200)
h3 = (200, 1)
W1 = (200, 200)
W2 = (200, 200)
W3 = (200, 1)
Epoch 2/20
1000/1000 [==============================] - 1s 541us/step - loss: 29789754.6150
on_epoch_end
h_local1_weight = (200, 2, 1)
h_local2_weight = (200, 1, 1)
h0 = (200, 2)
h0_abs = (200, 2)
h1 = (200, 200)
h2 = (200, 200)
h3 = (200, 1)
W1 = (200, 200)
W2 = (200, 200)
W3 = (200, 1)
Epoch 3/20
1000/1000 [==============================] - 1s 551us/

In [5]:
#Apply FDR control to DNN result
from DL.FDR.FDR_control import FDR_control;
control = FDR_control();
selected_features = control.controlFilter(data_folder_path +os.path.sep+ X_file_name, "/ihome/hpark/zhf16/causalDeepVASE/data/DNN_result", offset=1, q=0.05);
#Save the selected associations
selected_associations = [];
for ele in selected_features:
    selected_associations.append({"Feature1":ele,"Feature2":"Y"});
pd.DataFrame(selected_associations).to_csv("data/DNN_selected_associations.csv")

__init__
400


In [ ]:
#Run DG
#Load data
X_data = pd.read_csv("X_n1000_p50_rep20.csv");
# X_data
Y_data = pd.read_csv('y_si_n1000_p50_rep20.csv');
#Merge X and Y
dataset = pd.concat([X_data, Y_data], axis=1, join='inner');
print(dataset.shape);

#Calculate the covariance matrix
cov_mat = dataset.cov();
corr_inv = np.linalg.inv(cov_mat)
corr_inv = pd.DataFrame(data=corr_inv, index=cov_mat.index,columns=cov_mat.columns)
# corr_inv.head(2)

#Convert the columns to their numerical representations
col_map = {};
col_map_rev = {};
col_list = dataset.columns.to_list();
for index,ele in enumerate(col_list):
    col_map[ele] = index;
    col_map_rev[index] = ele;
print(dataset.shape);

#https://stats.stackexchange.com/questions/13810/threshold-for-correlation-coefficient-to-indicate-statistical-significance-of-a
# t = dataset.shape[0]**(1/2)

#The data may need to be normalized if neccessary.
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler();
# scaled_values = scaler.fit_transform(dataset);
# dataset.loc[:,:] = scaled_values;

#Initialize DG object
from causal.DegenerateGaussianScore import DegenerateGaussianScore
dg = DegenerateGaussianScore(dataset,discrete_threshold=0.2);

In [ ]:
selected_associations_sum = [];
#Load both MGM-identified and DNN associations
MGM_associations = pd.read_csv("X_n1000_p50_rep20_MGM_associations.csv");
for index,row in MGM_associations.iterrows():
    if row["Feature1"]=="Y" or row["Feature2"]=="Y":
        print("Found.");
        selected_associations_sum.append({"Feature1":row["Feature1"],"Feature2":row["Feature2"]});
        
DNN_associations = pd.read_csv("DNN_selected_associations.csv");
for index,row in DNN_associations.iterrows():
    selected_associations_sum.append({"Feature1":row["Feature1"],"Feature2":row["Feature2"]});

In [ ]:
for ele in selected_associations_sum:
    f1 = ele["Feature1"];
    f2 = ele["Feature2"];
    
    inv_val = abs(corr_inv[f1][f2]);
    if inv_val<0.0:
        continue;
    
    n1_idx = col_map[f1];
    n2_idx = col_map[f2];
    
    s1 = dg.localScore(n1_idx,{n2_idx});
    s2 = dg.localScore(n2_idx,{n1_idx});
    
    if s1<s2:
        print("Cause: "+f2+", Effect: "+f1);
    elif s1>s2:
        print("Cause: "+f1+", Effect: "+f2);
    else:
        print("Same score.");